# Football Leagues Analysis Last Seasson

Football Leagues analysis is going to be performed as follows:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get one or multiple **random samples** from the data set to better understand what the data is all about
  3. Identify **data entities**, **metrics** and **dimensions**
  4. **Columns/fields categorization**
4. Columns groups **basic profiling** to better understand our data set:
  1. **Timing related** columns basic profiling
  2. **Matches related** columns basic profiling
5. **Answer some business questions** to improve service
  1. **Ratio of wins/draws/defeats** games by type of game
  2. **Statistics about goals** comparing Den Haag with the rest of the league
  3. **Cumulative points** in time on different seassons
  4. **List of Leagues and Teams** to focus on to sign new players

Let's go for it:

## 1. PySpark environment setup

In [1]:
# Find where Spark is
import findspark
findspark.init()

# Get access to SparkSession: Where our user is gonna interact with in order to execute
# the analysis of the cluster.

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Data source and Spark data abstraction (DataFrame) setup

In [2]:
footballDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("football_data.csv")
#                 .option("delimiter",";") \


## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame

In [3]:
from IPython.display import display, Markdown

# Tell Spark to print Schema
footballDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % footballDF.count()))

root
 |-- Season: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Div: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- League: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTHG: integer (nullable = true)
 |-- FTAG: integer (nullable = true)
 |-- FTR: string (nullable = true)
 |-- HTHG: integer (nullable = true)
 |-- HTAG: integer (nullable = true)
 |-- HTR: string (nullable = true)
 |-- PSH: double (nullable = true)
 |-- PSD: double (nullable = true)
 |-- PSA: double (nullable = true)
 |-- B365H: double (nullable = true)
 |-- B365D: double (nullable = true)
 |-- B365A: double (nullable = true)
 |-- LBH: double (nullable = true)
 |-- LBD: double (nullable = true)
 |-- LBA: double (nullable = true)
 |-- BWH: double (nullable = true)
 |-- BWD: double (nullable = true)
 |-- BWA: double (nullable = true)
 |-- ABP: double (nullable = true)
 |-- AC

This DataFrame has **179807 rows**.

### B. Get one or multiple random samples from the data set

In [4]:
footballDF.cache() # optimization to make the processing faster
footballDF.sample(False, 0.1).take(2)

[Row(Season='2020', Datetime='2020-08-21 00:30:00', Div='USA', Country='USA', League='MLS', Referee=None, HomeTeam='New England Revolution', AwayTeam='Philadelphia Union', FTHG=0, FTAG=0, FTR='D', HTHG=-1, HTAG=-1, HTR=None, PSH=None, PSD=None, PSA=None, B365H=None, B365D=None, B365A=None, LBH=None, LBD=None, LBA=None, BWH=None, BWD=None, BWA=None, ABP=None, AC=None, AF=None, AFKC=None, AHCh=None, AHW=None, AHh=None, AO=None, AR=None, AS=None, AST=None, AT=None, AY=None, Attendance=None, Avg<2.5=None, Avg>2.5=None, AvgA=3.07, AvgAHA=None, AvgAHH=None, AvgC<2.5=None, AvgC>2.5=None, AvgCA=None, AvgCAHA=None, AvgCAHH=None, AvgCD=None, AvgCH=None, AvgD=3.62, AvgH=2.18, B365<2.5=None, B365>2.5=None, B365AH=None, B365AHA=None, B365AHH=None, B365C<2.5=None, B365C>2.5=None, B365CA=None, B365CAHA=None, B365CAHH=None, B365CD=None, B365CH=None, BSA=None, BSD=None, BSH=None, BWCA=None, BWCD=None, BWCH=None, Bb1X2=None, BbAH=None, BbAHh=None, BbAv<2.5=None, BbAv>2.5=None, BbAvA=None, BbAvAHA=None, 

### We need to modify some columns to be able to work with them, as they have special characters
   - "<" will be translated into "_"
   - ">" will be removed

In [5]:
footballDF = footballDF.withColumnRenamed("Avg<2.5","Avg_25").withColumnRenamed("B365<2.5","B365_25").withColumnRenamed("AvgC<2.5","AvgC_25")\
                  .withColumnRenamed("BbAv<2.5","BbAv_25").withColumnRenamed("BbMx<2.5","BbMx_25").withColumnRenamed("GB<2.5","GB_25")\
                  .withColumnRenamed("max<2.5","max_25").withColumnRenamed("P<2.5","P_25").withColumnRenamed("B365C<2.5","B365C_25")\
                  .withColumnRenamed("maxC<2.5","maxC_25").withColumnRenamed("PC<2.5","PC_25")\
                  .withColumnRenamed("Avg>2.5","Avg25m").withColumnRenamed("B365>2.5","B36525m").withColumnRenamed("AvgC>2.5","AvgC_25m")\
                  .withColumnRenamed("BbAv>2.5","BbAv25m").withColumnRenamed("BbMx>2.5","BbMx25m").withColumnRenamed("GB>2.5","GB25m")\
                  .withColumnRenamed("max>2.5","max25m").withColumnRenamed("P>2.5","P25m").withColumnRenamed("B365C>2.5","B365C25m")\
                  .withColumnRenamed("maxC>2.5","maxC25m").withColumnRenamed("PC>2.5","PC25m")
                 
footballDF.printSchema()

root
 |-- Season: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Div: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- League: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTHG: integer (nullable = true)
 |-- FTAG: integer (nullable = true)
 |-- FTR: string (nullable = true)
 |-- HTHG: integer (nullable = true)
 |-- HTAG: integer (nullable = true)
 |-- HTR: string (nullable = true)
 |-- PSH: double (nullable = true)
 |-- PSD: double (nullable = true)
 |-- PSA: double (nullable = true)
 |-- B365H: double (nullable = true)
 |-- B365D: double (nullable = true)
 |-- B365A: double (nullable = true)
 |-- LBH: double (nullable = true)
 |-- LBD: double (nullable = true)
 |-- LBA: double (nullable = true)
 |-- BWH: double (nullable = true)
 |-- BWD: double (nullable = true)
 |-- BWA: double (nullable = true)
 |-- ABP: double (nullable = true)
 |-- AC

### C. Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** Football matches (main one which is measured - facts), Results (dimension), Goals (dimension)
* **Metrics:** Datetime
* **Dimensions:** Home and Away Games, Ocassions, Faults ...

### D. Column categorization

The following could be a potential column categorization (The names do not match with the labeling that is given in the dataset, but will be simplified to better understand the content of the dataset):

* **Timing related columns:** *Year*, *Month*, *DayofMonth*, *DayOfWeek*
* **Matches related columns:** *Referee*, *HomeTeam*, *AwayTeam*, *Goals(Both)*, *Ocassions*, *Faults*, *Bets*

## 4. Columns groups basic profiling to better understand our data set
### A. Timing related columns basic profiling

In [6]:
## this is mine

from IPython.display import display, Markdown
#from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit
from pyspark.sql.functions import *



print ("Summary of columns Year, Month, DayofMonth and DayOfWeek:")
footballDF.select("Datetime",year("Datetime").alias("Year"),month("Datetime").alias("Month"),dayofmonth("Datetime").alias("DayMonth"),dayofweek("Datetime").alias("DayWeek")).summary().show()


print("Checking for nulls on columns Datetime,Division, Country,League, Referee, HomeTeam, AwayTeam:")
footballDF.select([count(when(col(c).isNull(), c)).alias(c) for c in footballDF.columns[1:10]]).show()


print("Checking for nulls on columns Ocassions, Faults: I realized that I am missing many information for minor leagues")
footballDF.select([count(when(col(c).isNull(), c)).alias(c) for c in footballDF.columns[103:113]]).show()

print("Checking amount of distinct values in columns Year, Month, DayofMonth and DayOfWeek:")
footballDF.select([countDistinct(c).alias(c) for c in ["Datetime"]]).show()

print ("Most and least frequent occurrences for DayofMonth and DayOfWeek columns:")

footballdatesDF = \
  footballDF.withColumn("Month", lit(month("Datetime"))).withColumn("Year", lit(year("Datetime")))\
            .withColumn("DayofMonth", lit(dayofmonth("Datetime"))).withColumn("dayofweek", date_format("Datetime", 'EEEE'))

dayofMonthOccurrencesDF = footballdatesDF.groupBy("DayofMonth").agg(count(lit(1)).alias("Total"))
dayOfWeekDF = footballdatesDF.groupBy("DayOfWeek").agg(count(lit(1)).alias("Total"))

leastFreqDayOfMonth    = dayofMonthOccurrencesDF.orderBy(col("Total").asc()).first()
mostFreqDayOfMonth     = dayofMonthOccurrencesDF.orderBy(col("Total").desc()).first()
leastFreqDayOfWeek     = dayOfWeekDF.orderBy(col("Total").asc()).first()
mostFreqDayOfWeek      = dayOfWeekDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqDayOfMonth", "mostFreqDayOfMonth", "leastFreqDayOfWeek", "mostFreqDayOfWeek", \
       "%d (%d occurrences)" % (leastFreqDayOfMonth["DayofMonth"], leastFreqDayOfMonth["Total"]), \
       "%d (%d occurrences)" % (mostFreqDayOfMonth["DayofMonth"], mostFreqDayOfMonth["Total"]), \
       "%s (%d occurrences)" % (leastFreqDayOfWeek["DayOfWeek"], leastFreqDayOfWeek["Total"]), \
       "%s (%d occurrences)" % (mostFreqDayOfWeek["DayOfWeek"], mostFreqDayOfWeek["Total"]))))



Summary of columns Year, Month, DayofMonth and DayOfWeek:
+-------+-------------------+-----------------+------------------+-----------------+-----------------+
|summary|           Datetime|             Year|             Month|         DayMonth|          DayWeek|
+-------+-------------------+-----------------+------------------+-----------------+-----------------+
|  count|             179807|           179807|            179807|           179807|           179807|
|   mean|               null|2011.423203768485| 6.624119194469626|15.96488457067856|4.607367900026139|
| stddev|               null|5.613329446715722|3.6382043853469965|8.816593683399846|2.630203884858417|
|    min|2000-07-28 00:00:00|             2000|                 1|                1|                1|
|    25%|               null|             2007|                 3|                8|                1|
|    50%|               null|             2012|                 8|               16|                6|
|    75%|      


| leastFreqDayOfMonth | mostFreqDayOfMonth | leastFreqDayOfWeek | mostFreqDayOfWeek |
|----|----|----|----|
| 31 (3128 occurrences) | 26 (6906 occurrences) | Thursday (2910 occurrences) | Saturday (84163 occurrences) |


### B. Matches related columns basic profiling

In [7]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of Goals, to see if there are inconsistencies:")
footballDF.select("FTHG","FTAG","HTHG","HTAG").summary().show()
print ("Summary of Seasons, to see if there are inconsistencies:")
footballDF.select("Season","League").summary().show()

print("Checking for nulls on columns regarding the Referee, Attendance, League, Division and shots:")
footballDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Season", "League", "Div", "Referee", "Attendance","HS","AS","HST","AST"]]).show()

print("Check which Leagues do not have -1 as goals to further work with them:")
footballDF.select("League").distinct().where(col("HTHG")!=-1).show(25,False)
print("Checking amount of distinct values in columns  regarding the Referee, Attendance, League, Division and shots:")
footballDF.select([countDistinct(c).alias(c) for c in ["Season", "League", "Div", "Referee", "Attendance","HS","AS","HST","AST"]]).show()



Summary of Goals, to see if there are inconsistencies:
+-------+------------------+------------------+------------------+-------------------+
|summary|              FTHG|              FTAG|              HTHG|               HTAG|
+-------+------------------+------------------+------------------+-------------------+
|  count|            179807|            179807|            179807|             179807|
|   mean|1.4928729137352827|1.1359290795130335|0.3404594926782606|0.20761149454693087|
| stddev|1.2582257000986128|1.1059134686363807|0.9783511708008087|  0.865702201578181|
|    min|                 0|                 0|                -1|                 -1|
|    25%|                 1|                 0|                 0|                  0|
|    50%|                 1|                 1|                 0|                  0|
|    75%|                 2|                 2|                 1|                  1|
|    max|                10|                 9|                 7|         

## 5. Answer some business questions to improve service


In [8]:
### THIS CHUNK WAS USED TO CHECK THAT THE ANALYSIS WAS CORRECTLY DONE WITH SQL SYNTAX

# footballDF.createOrReplaceTempView("footballtable")

# spark.sql("""
    
#     SELECT CATEGORIES,SUM(TRIAL1),(SELECT COUNT(*) FROM footballtable where HomeTeam = 'Den Haag' OR AwayTeam = 'Den Haag') as shit1,COUNT(*),SUM(TRIAL1)/(SELECT COUNT(*) FROM footballtable where HomeTeam = 'Den Haag' OR AwayTeam = 'Den Haag') as shit2 FROM(
#     SELECT case when (FTHG + FTAG >6) AND (HTHG + HTAG >4) AND (HTHG + HTAG < (FTHG + FTAG-1)) then 'CAT1'
#                         when (FTHG + FTAG >6) AND (HTHG + HTAG <4) then 'CAT2'
#                         else 'CAT3' end AS CATEGORIES, case when((HomeTeam = 'Den Haag'AND FTR ='H') OR (AwayTeam = 'Den Haag' AND FTR='A')) then 1 else 0 end AS TRIAL1

#     FROM footballtable
#     WHERE HomeTeam = 'Den Haag' OR AwayTeam = 'Den Haag'
#     )
#     GROUP BY CATEGORIES
#     LIMIT 10
# """).show()             

* We are going to work with **Den Haag** of the **Eredivisie** league in **Netherlands** as our main team, as Eredivise is not one of the main leagues and we need a team with non negative goals in the database to run our study

In [9]:
footballDF.where(col("League")=="Eredivisie").select("HomeTeam").distinct().show(40,False)

+----------------+
|HomeTeam        |
+----------------+
|Twente          |
|Cambuur         |
|Graafschap      |
|Roosendaal      |
|NAC Breda       |
|FC Emmen        |
|Ajax            |
|Willem II       |
|Go Ahead Eagles |
|AZ Alkmaar      |
|Roda            |
|Heerenveen      |
|Den Haag        |
|Volendam        |
|Heracles        |
|Excelsior       |
|Nijmegen        |
|Dordrecht       |
|PSV Eindhoven   |
|Feyenoord       |
|VVV Venlo       |
|For Sittard     |
|Sparta Rotterdam|
|Utrecht         |
|Den Bosch       |
|Zwolle          |
|Groningen       |
|Roda JC         |
|Vitesse         |
|Waalwijk        |
|Sparta          |
+----------------+



* We make sure that we are introducing correctly the names of the Seasons set in the database. We are interested in the 2 Seassons previous to the current one.

In [10]:
footballDF_2017 = footballDF.select(col("Season")).distinct().show()
footballDF_2017 = footballDF.where(col("League")=="Eredivisie").select("Season").distinct().show()

+---------+
|   Season|
+---------+
|     2016|
|     2020|
|     2012|
|2016/2017|
|2001/2002|
|2004/2005|
|     2019|
|     2017|
|     2014|
|     2013|
|2000/2001|
|2020/2021|
|2003/2004|
|2002/2003|
|2015/2016|
|     2018|
|2012/2013|
|2019/2020|
|2010/2011|
|2005/2006|
+---------+
only showing top 20 rows

+---------+
|   Season|
+---------+
|2016/2017|
|2001/2002|
|2004/2005|
|2000/2001|
|2003/2004|
|2002/2003|
|2015/2016|
|2012/2013|
|2019/2020|
|2010/2011|
|2005/2006|
|2017/2018|
|2007/2008|
|2006/2007|
|2014/2015|
|2008/2009|
|2013/2014|
|2011/2012|
|2009/2010|
|2018/2019|
+---------+



### A. Ratio of wins/draws/defeats games by type of game

In [11]:
# 1. We need to categorize the matches of the last 2 seasons to point out the main ones in which the team
#    is involved in the leaghe

# One way to visualize it is by comparing the two last seassons, where the team dropped in the ranking 
# considerably.

from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, sum, abs

totalgames = footballDF.count()
totalgames_cor = footballDF.count()

# We name footballDF_2017 to the seasons after 2017

footballDF_2017=footballDF.where((col("Season")=="2018/2019")|(col("Season")=="2019/2020"))

# We create a dF to the seasons after 2017 whith a new column "Categories" and another TeamSelected",
# which shows those matches that the team won

insideDFW = footballDF_2017 \
            .where((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag")) \
            .withColumn("Categories",when((col("FTHG")+ col("FTAG") > 6) & ((col("FTHG") - col("HTHG") + col("FTAG") - col("HTAG")) > 4),"Many Goals Game 2nd") \
                       .when((col("FTHG")+ col("FTAG") > 6) & ((col("HTHG")+ col("HTAG"))> 4),"Many Goals Game 1st") \
                       .when((col("FTHG")+col("FTAG"))==0,"No Goal Game") \
                       .when((col("FTHG")+ col("FTAG") > 6),"Many Goals Game") \
                       .when(((col("FTHG")+ col("FTAG")).between(4,6)) & ((col("HTHG") + col("HTAG")) >= 3) & (abs(col("HTHG") + col("HTAG")) > 1)& ((col("FTR") != "D")),"Resolved in 1st part") \
                       .when(((col("FTHG")+ col("FTAG")).between(4,6)) & ((col("FTHG")-col("HTHG") + col("FTAG")-col("HTAG")) >= 3) & (abs(col("FTHG")-col("HTHG") - (col("FTAG")-col("HTAG"))) > 1)& ((col("FTR") != "D")),"Resolved in 2nd part") \
                       .when(((col("FTHG")+ col("FTAG")).between(1,3)) & ((abs(col("HTHG") - col("HTAG")) == 1) |(abs(col("FTHG")-col("HTHG") - (col("FTAG")-col("HTAG"))) == 1)),"Tight Game") \
                       .otherwise("Other Games")
                       ) \
            .withColumn("TeamSelected",when(((col("HomeTeam")=="Den Haag")& (col("FTR")=="H"))|((col("AwayTeam") =="Den Haag")& (col("FTR")=="A")),1)\
                                            .otherwise(0))\
            .select("Categories","HomeTeam","AwayTeam","TeamSelected","Season")

insideDFW.cache()   # optimization to make the processing faster

insideDFW.show(10,False)

# We compute the ratio of Wins for each category.

inside2W = insideDFW \
    .where((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag")) \
    .select("Categories","TeamSelected","Season") \
    .groupBy("Categories","Season")\
    .agg(sum("TeamSelected").alias("Wins"),\
        count("TeamSelected").alias("Number of games"),\
        (sum("TeamSelected")/count("TeamSelected")).alias("Ratio of Wins"))\
    .orderBy("Categories")

inside2W.show(16)

# We repeat the previous steps as commented before but now for Draws

insideDFD = footballDF_2017 \
            .where((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag")) \
            .withColumn("Categories",when((col("FTHG")+ col("FTAG") > 6) & ((col("FTHG") - col("HTHG") + col("FTAG") - col("HTAG")) > 4),"Many Goals Game 2nd") \
                       .when((col("FTHG")+ col("FTAG") > 6) & ((col("HTHG")+ col("HTAG"))> 4),"Many Goals Game 1st") \
                       .when((col("FTHG")+col("FTAG"))==0,"No Goal Game") \
                       .when((col("FTHG")+ col("FTAG") > 6),"Many Goals Game") \
                       .when(((col("FTHG")+ col("FTAG")).between(4,6)) & ((col("HTHG") + col("HTAG")) >= 3) & (abs(col("HTHG") + col("HTAG")) > 1)& ((col("FTR") != "D")),"Resolved in 1st part") \
                       .when(((col("FTHG")+ col("FTAG")).between(4,6)) & ((col("FTHG")-col("HTHG") + col("FTAG")-col("HTAG")) >= 3) & (abs(col("FTHG")-col("HTHG") - (col("FTAG")-col("HTAG"))) > 1)& ((col("FTR") != "D")),"Resolved in 2nd part") \
                       .when(((col("FTHG")+ col("FTAG")).between(1,3)) & ((abs(col("HTHG") - col("HTAG")) == 1) |(abs(col("FTHG")-col("HTHG") - (col("FTAG")-col("HTAG"))) == 1)),"Tight Game") \
                       .otherwise("Other Games")
                       ) \
            .withColumn("TeamSelected",when(((col("HomeTeam")=="Den Haag")& (col("FTR")=="D"))|((col("AwayTeam") =="Den Haag")& (col("FTR")=="D")),1)\
                                            .otherwise(0))\
            .select("Categories","HomeTeam","AwayTeam","TeamSelected","Season")

insideDFD.cache()   # optimization to make the processing faster



inside2D = insideDFD \
    .where((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag")) \
    .select("Categories","TeamSelected","Season") \
    .groupBy("Categories","Season")\
    .agg(sum("TeamSelected").alias("Draws"),\
        count("TeamSelected").alias("Number of games"),\
        (sum("TeamSelected")/count("TeamSelected")).alias("Ratio of Draws"))\
    .orderBy("Categories")
inside2D.show(16)

# We repeat the previous steps as commented before but now for Defeats


insideDFL = footballDF_2017 \
            .where((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag")) \
            .withColumn("Categories",when((col("FTHG")+ col("FTAG") > 6) & ((col("FTHG") - col("HTHG") + col("FTAG") - col("HTAG")) > 4),"Many Goals Game 2nd") \
                       .when((col("FTHG")+ col("FTAG") > 6) & ((col("HTHG")+ col("HTAG"))> 4),"Many Goals Game 1st") \
                       .when((col("FTHG")+col("FTAG"))==0,"No Goal Game") \
                       .when((col("FTHG")+ col("FTAG") > 6),"Many Goals Game") \
                       .when(((col("FTHG")+ col("FTAG")).between(4,6)) & ((col("HTHG") + col("HTAG")) >= 3) & (abs(col("HTHG") + col("HTAG")) > 1)& ((col("FTR") != "D")),"Resolved in 1st part") \
                       .when(((col("FTHG")+ col("FTAG")).between(4,6)) & ((col("FTHG")-col("HTHG") + col("FTAG")-col("HTAG")) >= 3) & (abs(col("FTHG")-col("HTHG") - (col("FTAG")-col("HTAG"))) > 1)& ((col("FTR") != "D")),"Resolved in 2nd part") \
                       .when(((col("FTHG")+ col("FTAG")).between(1,3)) & ((abs(col("HTHG") - col("HTAG")) == 1) |(abs(col("FTHG")-col("HTHG") - (col("FTAG")-col("HTAG"))) == 1)),"Tight Game") \
                       .otherwise("Other Games")
                       ) \
            .withColumn("TeamSelected",when(((col("HomeTeam")=="Den Haag")& (col("FTR")=="A"))|((col("AwayTeam") =="Den Haag")& (col("FTR")=="H")),1)\
                                            .otherwise(0))\
            .withColumn("Location",when(col("HomeTeam")=="Den Haag","Home")\
                                  .when(col("AwayTeam")=="Den Haag","Away"))\
            .select("Categories","HomeTeam","AwayTeam","TeamSelected","Season","Location","TeamSelected")

insideDFL.cache()   # optimization to make the processing faster

inside2L = insideDFL \
    .where((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag")) \
    .select("Categories","TeamSelected","Season") \
    .groupBy("Categories","Season")\
    .agg(sum("TeamSelected").alias("Defeats"),\
        count("TeamSelected").alias("Number of games"),\
        (sum("TeamSelected")/count("TeamSelected")).alias("Ratio of Defeats"))\
    .orderBy("Categories")
inside2L.show(16)


# Takeaways:
    # In this last seassons, there is no predominancy of a category over the others. (Others and Tight)
    # The team played less games that were resolved in the 1st part (Besides not winning them).
    # The moral of the team was undermined with adverse situations, and did not know how to overcome them)
    # The team encountered more Tight games that did not know how to resolve
    # We can have another look now to determine from those tight games, to further analyze home and away.

display(Markdown("Let's take a closer look into **Tight Games**:"))

# We will group by Location (Home or Away game) to see the differences
insideDFL \
    .where((col("Categories")=="Tight Game")&((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag"))) \
    .select("Categories","Season","Location","TeamSelected")\
    .groupby("Season","Location")\
    .agg(sum("TeamSelected").alias("Defeats"), count("TeamSelected").alias("Number of games"),\
        ((sum("TeamSelected")/count("TeamSelected"))).alias("Ratio of Defeats"))\
    .orderBy("Location")\
    .show()

# Specially, the team benefited on the last Seasson from tight games away from their stadium. New strategies
# could involve pressing more the rivals on critical moments of the game. Further analysis could be included
# with information of the minutes when the goals took place on each match.

+--------------------+----------------+-------------+------------+---------+
|Categories          |HomeTeam        |AwayTeam     |TeamSelected|Season   |
+--------------------+----------------+-------------+------------+---------+
|Other Games         |AZ Alkmaar      |Den Haag     |0           |2019/2020|
|No Goal Game        |Den Haag        |Heracles     |0           |2019/2020|
|Other Games         |Heerenveen      |Den Haag     |0           |2019/2020|
|Tight Game          |Den Haag        |PSV Eindhoven|0           |2019/2020|
|Resolved in 1st part|Sparta Rotterdam|Den Haag     |0           |2019/2020|
|No Goal Game        |Den Haag        |Vitesse      |0           |2019/2020|
|Resolved in 2nd part|Utrecht         |Den Haag     |0           |2019/2020|
|Tight Game          |Den Haag        |Waalwijk     |1           |2019/2020|
|Many Goals Game     |Ajax            |Den Haag     |0           |2019/2020|
|Other Games         |Den Haag        |Groningen    |0           |2019/2020|

Let's take a closer look into **Tight Games**:

+---------+--------+-------+---------------+------------------+
|   Season|Location|Defeats|Number of games|  Ratio of Defeats|
+---------+--------+-------+---------------+------------------+
|2018/2019|    Away|      3|              8|             0.375|
|2019/2020|    Away|      2|              3|0.6666666666666666|
|2019/2020|    Home|      4|              6|0.6666666666666666|
|2018/2019|    Home|      2|              4|               0.5|
+---------+--------+-------+---------------+------------------+



### B. Statistics about goals comparing Den Haag with the rest of the league

In [12]:
# We create a dataframe for the current season
footballDF_2019=footballDF.where((col("Season")=="2019/2020"))

# We define a new dataframe with the points obtained by the team in each match 
insideDF2 = footballDF_2019 \
            .where(((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag"))) \
            .withColumn("Pointsgame", when(((col("HomeTeam")=="Den Haag")& (col("FTR")=="H"))|((col("AwayTeam") =="Den Haag")& (col("FTR")=="A")),3) \
            .when(((col("HomeTeam")=="Den Haag") & (col("FTR")=="A")) | ((col("AwayTeam") =="Den Haag") & (col("FTR") == "H")),0) \
            .otherwise(1)) \
            .select("Datetime","Div","FTHG","FTAG","League","Hometeam","AwayTeam","Pointsgame")
            

insideDF2.cache()   # optimization to make the processing faster

insideDF2.show(50,False)


# We make sure that the League that we are considering only exists in Netherlands
insideDF2.select("Div","League").distinct().show(30, False) 

# We compute the statistics of the game for games played at Home
inside2 = insideDF2 \
    .where(col("HomeTeam")=="Den Haag") \
    .agg(sum("Pointsgame").alias("Points"),\
     avg("FTHG").alias("AVG Home Goals_T"), \
     stddev("FTHG").alias("STDEV Home Goals_T"), \
     avg("FTAG").alias("AVG Away Goals"), \
     stddev("FTAG").alias("STDEV Away Goals"))

print("The AVG Home Goals_T means avg goals of Den Haag at home, while Avg Away Goals mean the goals they received at home")
inside2.show(5)

## In this case we are going to show the statistics of the average of the rest of the teams in the league
## to compare with

insideDF3 = footballDF_2019 \
            .where((col("HomeTeam")!="Den Haag") & (col("Div")=="N1") & (col("AwayTeam")!="Den Haag")) \
            .select("Datetime","Div","FTHG","FTAG","League","Hometeam","AwayTeam")
            
inside3 = insideDF3 \
    .where((col("HomeTeam")!="Den Haag") & (col("Div")=="N1") & (col("AwayTeam")!="Den Haag") )\
    .agg(avg("FTHG").alias("AVG Home Goals Rest"), \
     stddev("FTHG").alias("STDEV Home Goals Rest"), \
     avg("FTAG").alias("AVG Away Goals Rest"), \
     stddev("FTAG").alias("STDEV Away Goals Rest"))
inside3.show(5)

# We compute the statistics of the game for games played Away

inside2 = insideDF2 \
    .where(col("AwayTeam")=="Den Haag") \
    .agg(sum("Pointsgame").alias("Points"),\
     mean("FTHG").alias("AVG Home Goals"), \
     stddev("FTHG").alias("STDEV Home Goals"), \
     mean("FTAG").alias("AVG Away Goals_T"), \
     stddev("FTAG").alias("STDEV Away Goals_T"))

print("The AVG Away Goals_T means avg goals of Den Haag away, while Avg Home Goals mean the goals they received away")
inside2.show(5)


## We can see that in Away matches, they receive much more goals than the average of the league
# Also in home, they are not too aggressive, since they do score half of the goals as the average of the
#teams

+-------------------+---+----+----+----------+----------------+----------------+----------+
|Datetime           |Div|FTHG|FTAG|League    |Hometeam        |AwayTeam        |Pointsgame|
+-------------------+---+----+----+----------+----------------+----------------+----------+
|2020-03-07 17:30:00|N1 |4   |0   |Eredivisie|AZ Alkmaar      |Den Haag        |0         |
|2020-02-29 17:30:00|N1 |0   |0   |Eredivisie|Den Haag        |Heracles        |1         |
|2020-02-22 18:45:00|N1 |2   |2   |Eredivisie|Heerenveen      |Den Haag        |1         |
|2020-02-15 18:45:00|N1 |0   |3   |Eredivisie|Den Haag        |PSV Eindhoven   |0         |
|2020-02-11 19:45:00|N1 |4   |2   |Eredivisie|Sparta Rotterdam|Den Haag        |0         |
|2020-02-01 17:30:00|N1 |0   |0   |Eredivisie|Den Haag        |Vitesse         |1         |
|2020-01-24 19:00:00|N1 |4   |0   |Eredivisie|Utrecht         |Den Haag        |0         |
|2020-01-19 15:45:00|N1 |2   |0   |Eredivisie|Den Haag        |Waalwijk        |

We can also observe the individual statistics for each team, instead of on average

In [13]:
footballDF_2019.where(col("League")=="Eredivisie").groupby("HomeTeam").agg((sum("FTHG")/13).alias("Avg_HomeGoals"),stddev("FTHG").alias("STD_HomeGoals"),(sum("FTAG")/13).alias("Avg_AwayGoals"),stddev("FTAG").alias("STD_AwayGoals")).orderBy("Avg_HomeGoals").show()
footballDF_2019.where(col("League")=="Eredivisie").groupby("AwayTeam").agg((sum("FTAG")/13).alias("Avg_AwayGoals"),stddev("FTAG").alias("STD_AwayGoals"),(sum("FTHG")/13).alias("Avg_HomeGoals"),stddev("FTHG").alias("STD_HomeGoals")).orderBy("Avg_AwayGoals").show()

# Looking into the following table, we expose each teams' statistics of their games at home (Sorted by Avg_HomeGoals)
# Compared to other teams, Den Haag occupies the last position of the table in goals scored during last seasson
# with a huge difference. Besides, their numbers on goals received at home are too high

# The second table shows the teams' statistics of their games away (Sorted by AvgAwayGoals)
# The most surprising takeaway is that the team does not suffer attacking on these games, but defending
# the rivals. Since they score many goals, but they also receive a lot of goals.

+----------------+------------------+------------------+------------------+------------------+
|        HomeTeam|     Avg_HomeGoals|     STD_HomeGoals|     Avg_AwayGoals|     STD_AwayGoals|
+----------------+------------------+------------------+------------------+------------------+
|        Den Haag|0.8461538461538461|0.9870962335856491|1.3076923076923077|1.3774744634423888|
|        Waalwijk|               1.0|0.9128709291752769|1.5384615384615385| 1.126601424298216|
|       VVV Venlo|1.0769230769230769|0.9540735874430285|1.6153846153846154| 1.445594545418455|
|      Heerenveen|1.3076923076923077|0.6992932067530678|1.3846153846153846| 0.825420305855557|
|       Groningen|1.3076923076923077|0.9473309334313418|0.7692307692307693|0.9268086959962983|
|       Willem II|1.3076923076923077|1.2506408613597126|0.6923076923076923|0.6304251719561152|
|     For Sittard|1.6923076923076923|1.3155870289605438|1.3076923076923077|1.1821319289469756|
|          Twente|1.7692307692307692|1.23516841994

### C. Cumulative points in time on different seassons

In [14]:
from pyspark.sql import Window

import sys

from pyspark.sql.window import Window

import pyspark.sql.functions as f

# We can get the cum_sum of the points of the games to better understand when the team was not performing well.
cum_sum = insideDF2.orderBy("Datetime").withColumn('cumsum', f.sum(insideDF2.Pointsgame).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0)))
# cum_sum.show(40,False)

print("Check the time evolution of the points in Season 2019/2020")
cum_sum.select(month(col("Datetime")).alias("Month"),"Pointsgame","cumsum").groupby("Month").agg(f.max(col("cumsum")).alias("Cumsum Points")).show(25)

# We want to obtain the cumsum for each season, so we define the df for each year

footballDF_2018=footballDF.where((col("Season")=="2018/2019"))
insideDF2 = footballDF_2018 \
            .where(((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag"))) \
            .withColumn("Pointsgame", when(((col("HomeTeam")=="Den Haag")& (col("FTR")=="H"))|((col("AwayTeam") =="Den Haag")& (col("FTR")=="A")),3) \
            .when(((col("HomeTeam")=="Den Haag") & (col("FTR")=="A")) | ((col("AwayTeam") =="Den Haag") & (col("FTR") == "H")),0) \
            .otherwise(1)) \
            .select("Datetime","Div","FTHG","FTAG","League","Hometeam","AwayTeam","Pointsgame")


cum_sum = insideDF2.orderBy("Datetime").withColumn('cumsum', f.sum(insideDF2.Pointsgame).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0)))

# We use the function max, to determine the cummulative points at the end of each month
print("Check the time evolution of the points in Season 2018/2019")
cum_sum.select(month(col("Datetime")).alias("Month"),"Pointsgame","cumsum").groupby("Month").agg(f.max(col("cumsum")).alias("Cumsum Points")).show(25)


## The biggest difference was in the begining of Winter (Months of November and December,where the team
##  only got 2 points per month: Meaning 2 Draws / Month and the rest they lost them. This could be taken
## into account by the trainer to improve the trainings and demand more physical trainings, and rotate
## players much more.)

## To contrast even more the results, 2017 was plotted also to demonstrate that the team in March 2020
## before the break, due to COVID, had the same points as last 2 Seasons but in December.

footballDF_2018=footballDF.where((col("Season")=="2017/2018"))
insideDF2 = footballDF_2018 \
            .where(((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag"))) \
            .withColumn("Pointsgame", when(((col("HomeTeam")=="Den Haag")& (col("FTR")=="H"))|((col("AwayTeam") =="Den Haag")& (col("FTR")=="A")),3) \
            .when(((col("HomeTeam")=="Den Haag") & (col("FTR")=="A")) | ((col("AwayTeam") =="Den Haag") & (col("FTR") == "H")),0) \
            .otherwise(1)) \
            .select("Datetime","Div","FTHG","FTAG","League","Hometeam","AwayTeam","Pointsgame","Season")
            
cum_sum = insideDF2.orderBy("Datetime").withColumn('cumsum', f.sum(insideDF2.Pointsgame).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0)))

print("Check the time evolution of the points in Season 2017/2018")
cum_sum_global = cum_sum.select("Season",month(col("Datetime")).alias("Month"),"Pointsgame","cumsum").groupby("Season","Month").agg(f.max(col("cumsum")).alias("Cumsum Points"))
cum_sum_global.show()

Check the time evolution of the points in Season 2019/2020
+-----+-------------+
|Month|Cumsum Points|
+-----+-------------+
|    8|            6|
|    9|            6|
|   10|            9|
|   11|           11|
|   12|           13|
|    1|           16|
|    2|           19|
|    3|           19|
+-----+-------------+

Check the time evolution of the points in Season 2018/2019
+-----+-------------+
|Month|Cumsum Points|
+-----+-------------+
|    8|            3|
|    9|            8|
|   10|           12|
|   11|           15|
|   12|           20|
|    1|           21|
|    2|           27|
|    3|           29|
|    4|           39|
|    5|           45|
+-----+-------------+

Check the time evolution of the points in Season 2017/2018
+---------+-----+-------------+
|   Season|Month|Cumsum Points|
+---------+-----+-------------+
|2017/2018|    8|            0|
|2017/2018|    9|           10|
|2017/2018|   10|           13|
|2017/2018|   11|           17|
|2017/2018|   12|        

In [15]:
# We can observe the fluctuation of points for every seasson by pivoting the table grouped by Season 
# and Month

footballDF_2018=footballDF.where((col("Season")!="2021"))
insideDF2 = footballDF_2018 \
            .where(((col("HomeTeam") == "Den Haag") | (col("AwayTeam") == "Den Haag"))) \
            .withColumn("Pointsgame", when(((col("HomeTeam")=="Den Haag")& (col("FTR")=="H"))|((col("AwayTeam") =="Den Haag")& (col("FTR")=="A")),3) \
            .when(((col("HomeTeam")=="Den Haag") & (col("FTR")=="A")) | ((col("AwayTeam") =="Den Haag") & (col("FTR") == "H")),0) \
            .otherwise(1)) \
            .select("Datetime","Div","FTHG","FTAG","League","Hometeam","AwayTeam","Pointsgame","Season")
            
# cum_sum.show(40,False)
print("Check the time evolution of the points in Season 2017/2018")
cum_sum_global = insideDF2.select("Season",month(col("Datetime")).alias("Month"),"Pointsgame").groupby("Season","Month").agg(f.sum(col("Pointsgame")).alias("Sum Points")).orderBy("Season")
cum_sum_global.show()

## We can obtain the sum of points per year and month to observe more clearly the seasonality
cum_sum_global.groupBy("Season").pivot("Month").sum("Sum Points").orderBy("Season").show()

Check the time evolution of the points in Season 2017/2018
+---------+-----+----------+
|   Season|Month|Sum Points|
+---------+-----+----------+
|2003/2004|   12|         0|
|2003/2004|    4|         3|
|2003/2004|   11|         1|
|2003/2004|    8|         1|
|2003/2004|    3|         2|
|2003/2004|    5|         6|
|2003/2004|    2|        10|
|2003/2004|   10|         4|
|2003/2004|    1|         3|
|2003/2004|    9|         4|
|2004/2005|    5|         6|
|2004/2005|    8|         3|
|2004/2005|    9|         1|
|2004/2005|   10|         0|
|2004/2005|   12|         4|
|2004/2005|    4|         1|
|2004/2005|    1|         1|
|2004/2005|    3|         0|
|2004/2005|    2|         9|
|2005/2006|    4|         4|
+---------+-----+----------+
only showing top 20 rows

+---------+---+---+---+----+----+---+---+---+----+---+
|   Season|  1|  2|  3|   4|   5|  8|  9| 10|  11| 12|
+---------+---+---+---+----+----+---+---+---+----+---+
|2003/2004|  3| 10|  2|   3|   6|  1|  4|  4|   1|  0|

### D. List of Leagues and Teams to focus on, to sign new players

In [16]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, sum

#Check the name of first divisions for 4 Best leagues in Europe
footballdatesDF.where(col("Country").isin({"Spain","England","Germany","Italy"})).select("Div","Country","League").distinct().orderBy("Country").show(30,False)

# We filter to take out the first divisions of each of those leagues (Players too Expensive)

Minor_Leagues = footballdatesDF.filter(~footballdatesDF.Div.isin("E0","SP1","I1","D1"))

# We compute the table of Similar leagues with normalized goals/teams
Compare_Leagues = Minor_Leagues \
        .where(col("Year")==2019) \
        .withColumn("Total Goals", col("FTHG")+col("FTAG")) \
        .groupBy(col("League")) \
        .agg(sum(col("Total Goals")).alias("Number Goals"),\
             countDistinct("HomeTeam").alias("Number Teams"), first("Country")) \
        .withColumn("Goals/Teams", col("Number Goals")/col("Number Teams"))

Compare_Leagues.orderBy("Goals/Teams").show(45,False)

# We set our criteria of similarity: Goals/Games +- 6. The number 51.52 can be extracted from 
# Compare_leagues for Eredivisie
Leagues_Selected = Compare_Leagues \
                .where((col("Goals/Teams")< (51.52 + 6)) & (col("Goals/Teams") > (51.52 - 6))) #\
#                 .select("League").rdd.map(r => r(0)).collect()

## With a low level API I get here the list of leagues to be able to filter new DF to obtain the teams
Leagues_li = Leagues_Selected.rdd.map(lambda x: x.League).collect()





+---+-------+------------------------+
|Div|Country|League                  |
+---+-------+------------------------+
|E2 |England|League 1                |
|E1 |England|Championship            |
|EC |England|Conference              |
|E0 |England|Premier League          |
|E3 |England|League 2                |
|D1 |Germany|Bundesliga 1            |
|D2 |Germany|Bundesliga 2            |
|I2 |Italy  |Serie B                 |
|I1 |Italy  |Serie A                 |
|SP2|Spain  |La Liga Segunda Division|
|SP1|Spain  |La Liga Primera Division|
+---+-------+------------------------+

+------------------------+------------+------------+---------------------+------------------+
|League                  |Number Goals|Number Teams|first(Country, false)|Goals/Teams       |
+------------------------+------------+------------+---------------------+------------------+
|Primera Division        |746         |28          |Argentina            |26.642857142857142|
|Veikkausliiga           |409         

In [17]:
## Check the distinct Teams in those leagues (291)
Minor_Leagues.where(col("League").isin(Leagues_li)).select(col("HomeTeam")).distinct().head(30)
Minor_Leagues.where(col("League").isin(Leagues_li)).select(col("HomeTeam")).distinct().count()

241

In [18]:
#Make sure that Eredivisie is only in Netherlands

footballDF.where(col("League").isin("Eredivisie")).select(col("Country")).distinct().show()


+-----------+
|    Country|
+-----------+
|Netherlands|
+-----------+



In [19]:
# We make a list of Teams to analyze and we take out the Championship in England as it has teams of Premier League
Teams_Minor_Leagues_List = Minor_Leagues.where((col("League").isin(Leagues_li))&(col("League")!="Championship")).select(col("HomeTeam")).distinct().rdd.map(lambda x: x.HomeTeam).collect()

In [20]:
Teams_Minor_Leagues_List

['Twente',
 'Guiseley',
 'Cambuur',
 'Histon',
 'Salzburg',
 'Graafschap',
 'Roosendaal',
 'NAC Breda',
 'Toronto FC',
 'Canvey Island',
 'Chicago Fire',
 'Notts County',
 'Chesterfield',
 'Macclesfield',
 'Kidderminster',
 'Atlanta United',
 'Minnesota United',
 'Tamworth',
 'Scarborough',
 'Veracruz',
 'Rapid Vienna',
 'Wolfsberger AC',
 'Los Angeles Galaxy',
 'Oxford',
 'Newport County',
 'Toluca',
 'Atlanta Utd',
 'Cambridge',
 'U.N.A.M.- Pumas',
 'Cheltenham',
 'Chiapas',
 'FC Emmen',
 'Ajax',
 'Willem II',
 'Austria Vienna',
 'Woking',
 'Philadelphia Union',
 'Atl. San Luis',
 'Monarcas',
 'Go Ahead Eagles',
 'AZ Alkmaar',
 'Boreham Wood',
 'Hayes & Yeading',
 'Queretaro',
 'Gateshead',
 'Roda',
 'Stafford Rangers',
 'Morecambe',
 'Seattle Sounders',
 'Altrincham',
 'AFC Wimbledon',
 'Heerenveen',
 'Dorados de Sinaloa',
 'LASK',
 'Braintree Town',
 'Den Haag',
 'Eastleigh',
 'Leyton Orient',
 'FC Dallas',
 'Barnet',
 'Exeter',
 'Salford',
 'Mansfield',
 'Grodig',
 'Inter Miami',


In [21]:
## Checking if for those leagues the teams do not have Nulls in the Ocassions
Minor_Leagues.where(col("HomeTeam").isin(Teams_Minor_Leagues_List)).select("HomeTeam").where(col("HST").isNotNull()).distinct().show()

+--------------+
|      HomeTeam|
+--------------+
|        Twente|
|      Guiseley|
|        Histon|
|    Graafschap|
|     NAC Breda|
| Canvey Island|
|  Macclesfield|
|  Notts County|
|  Chesterfield|
| Kidderminster|
|      Tamworth|
|   Scarborough|
|        Oxford|
|Newport County|
|     Cambridge|
|    Cheltenham|
|      FC Emmen|
|          Ajax|
|     Willem II|
|        Woking|
+--------------+
only showing top 20 rows



In [22]:
# The first table shows how effective teams are regarding the goals vs ocassions at home.
# The second shows the same information but for teams playing away.

# Luton, Bradford, Exeter, Blackpool and Zwolle would be Teams to consider to look for precise strikers
# Southampton, Preston, Everton, Heerenveen and Accrington would be interesting teams to check for strikers
# who have good numbers in away matches (dealing with pressure)

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, sum, coalesce,avg

# Comment: We use coalesce to set to -1 whenever goals are not available in the dataset

footballDF_2019.where(col("HomeTeam").isin(Teams_Minor_Leagues_List)).withColumn("EffectivityHome",coalesce(col("FTHG")/col("HST"),lit(-1.0))).withColumn("EffectivityAway",coalesce(col("FTAG")/col("AST"),lit(-1.0))).groupby("HomeTeam").agg((avg("EffectivityHome").alias("AVG_Eff_Home")),(avg("EffectivityAway").alias("AVG_Eff_Away"))).orderBy(col("AVG_Eff_Home").desc()).show()

footballDF_2019.where(col("AwayTeam").isin(Teams_Minor_Leagues_List)).withColumn("EffectivityHome",coalesce(col("FTHG")/col("HST"),lit(-1.0))).withColumn("EffectivityAway",coalesce(col("FTAG")/col("AST"),lit(-1.0))).groupby("HomeTeam").agg((avg("EffectivityHome").alias("AVG_Eff_Home")),(avg("EffectivityAway").alias("AVG_Eff_Away"))).orderBy(col("AVG_Eff_Away").desc()).show()




+----------------+-------------------+--------------------+
|        HomeTeam|       AVG_Eff_Home|        AVG_Eff_Away|
+----------------+-------------------+--------------------+
|           Luton| 0.5072463768115941| 0.36143892339544514|
|          Exeter| 0.4800705467372134| 0.21296296296296294|
|          Zwolle|0.47182539682539676|  0.3974206349206349|
|         Utrecht|0.45515873015873015| 0.31646825396825395|
|Sparta Rotterdam|0.44273504273504266|  0.3194139194139194|
|          Twente| 0.4175824175824176| 0.26520979020979024|
|      Cheltenham| 0.4152116402116402|  0.3260582010582011|
|   Leyton Orient| 0.3843358395989975| 0.28439849624060143|
|      Accrington| 0.3810248574954458| 0.46176470588235285|
|     Bristol Rvs| 0.3805555555555556| 0.23703703703703705|
|        FC Emmen| 0.3753663003663003| 0.09386446886446885|
|         Vitesse| 0.3604700854700855|   0.257020757020757|
|        Heracles| 0.3567765567765568|  0.1727106227106227|
|          Burton|0.35387488328664807|  

In [23]:
footballDF_2019.where(col("HomeTeam").isin(Teams_Minor_Leagues_List)).withColumn("TacticalFoulsHome",coalesce((col("HY")+col("HR"))/col("HF"),lit(-1.0))).withColumn("TacticalFoulsAway",coalesce((col("AY")+col("AR"))/col("AF"),lit(-1.0))).groupby("HomeTeam").agg((avg("TacticalFoulsHome").alias("AVG_TacFoul_Home")),(avg("TacticalFoulsAway").alias("AVG_TacFoul_Away"))).orderBy(col("AVG_TacFoul_Home").desc()).show()

footballDF_2019.where(col("AwayTeam").isin(Teams_Minor_Leagues_List)).withColumn("TacticalFoulsHome",coalesce((col("HY")+col("HR"))/col("HF"),lit(-1.0))).withColumn("TacticalFoulsAway",coalesce((col("AY")+col("AR"))/col("AF"),lit(-1.0))).groupby("HomeTeam").agg((avg("TacticalFoulsHome").alias("AVG_TacFoul_Home")),(avg("TacticalFoulsAway").alias("AVG_TacFoul_Away"))).orderBy(col("AVG_TacFoul_Away").desc()).show()


## If we look into the tables, there are some potential Teams to go delve into their players, as they
# may have a good striker with a strong tactical Midfield commiting Faults but avoiding many cards.
# Those teams could be Zwolle, Bradford. Besides, Zwolle is a versatile team, figuring out in both tables
# in Home and Away Games. Potential players could be investigated by the club for next Seasson.



# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HFKC = Home Team Free Kicks Conceded
# AFKC = Away Team Free Kicks Conceded
# HO = Home Team Offsides
# AO = Away Team Offsides
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards
# HR = Home Team Red Cards
# AR = Away Team Red Cards

+--------------+-------------------+-------------------+
|      HomeTeam|   AVG_TacFoul_Home|   AVG_TacFoul_Away|
+--------------+-------------------+-------------------+
|   For Sittard|0.21903972096279786|0.19879891350479584|
|      Waalwijk|0.21339927503895112| 0.1517130305591844|
|  Forest Green|0.19321935273492366|0.15041517623509498|
|      FC Emmen|0.17739546778008317|0.18724622646027325|
|   Bristol Rvs|0.17333909735870517|0.12564594751379649|
|       Vitesse|0.17304834054834056|0.16619566331104793|
|     Groningen|0.16968063987294757|0.14480199287891596|
|    Accrington| 0.1678350603341498|0.11699209095402867|
|        Zwolle|0.16428571428571428|0.21659234012175185|
|      Den Haag| 0.1640059128508759|0.12638869426199745|
|     VVV Venlo| 0.1621305918817231|0.11686401205631974|
|  Macclesfield|0.16104162721809778|0.15792486022749183|
|     Feyenoord| 0.1581973581973582| 0.1848572995631819|
|       Lincoln|0.15651912586897107|0.14756208324547423|
|       Salford|0.1500960726579

In [24]:
# We can locate in which leagues and Divisions this team has played
footballDF.where(col("HomeTeam")=="Zwolle").select("League","Div").distinct().show()

+----------+---+
|    League|Div|
+----------+---+
|Eredivisie| N1|
+----------+---+



In [25]:
# We can locate in which leagues and Divisions this team has played

footballDF.where(col("HomeTeam")=="Bradford").select("League","Div").distinct().show()

# Since there are some leagues where they played, let's see this year to get an idea of the market value
# of their players

footballDF.where((col("HomeTeam")=="Bradford")&(col("Season")=="2019/2020")).select("League","Div").distinct().show()

# The team is in 3rd Division, and may have kept some players from the previous season, and their price
# may be not too high.

+--------------+---+
|        League|Div|
+--------------+---+
|      League 2| E3|
|      League 1| E2|
|  Championship| E1|
|Premier League| E0|
+--------------+---+

+--------+---+
|  League|Div|
+--------+---+
|League 2| E3|
+--------+---+



#### We could repeat the same steps, but setting the criteria of similarity to Nºgoals/games played


In [26]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, sum

#Check the name of first divisions for 4 Best leagues in Europe
footballdatesDF.where(col("Country").isin({"Spain","England","Germany","Italy"})).select("Div","Country","League").distinct().orderBy("Country").show(30,False)

# We filter to take out the first divisions of each of those leagues (Players too Expensive)

Minor_Leagues = footballdatesDF.filter(~footballdatesDF.Div.isin("E0","SP1","I1","D1"))

# We compute the table of Similar leagues with normalized goals/teams
Compare_Leagues = Minor_Leagues \
        .where(col("Year")==2019) \
        .withColumn("Total Goals", col("FTHG")+col("FTAG")) \
        .groupBy(col("League")) \
        .agg(sum(col("Total Goals")).alias("Number Goals"),\
             countDistinct("HomeTeam").alias("Number Teams"), first("Country")) \
        .withColumn("Goals/Games", col("Number Goals")/((col("Number Teams")-1)*(col("Number Teams"))))

Compare_Leagues.orderBy("Goals/Games").show(45,False)

# We set our criteria of similarity: Goals/Games +- 0.5  The number 2.5285... can be extracted from 
# Compare_leagues for Eredivisie
Leagues_Selected = Compare_Leagues \
                .where((col("Goals/Games")< (2.5285714285714285 + 0.5)) & (col("Goals/Games") > (2.5285714285714285 - 0.5))) #\
#                 .select("League").rdd.map(r => r(0)).collect()

## With a low level API I get here the list of leagues to be able to filter new DF to obtain the teams
Leagues_li = Leagues_Selected.rdd.map(lambda x: x.League).collect()





+---+-------+------------------------+
|Div|Country|League                  |
+---+-------+------------------------+
|E0 |England|Premier League          |
|EC |England|Conference              |
|E2 |England|League 1                |
|E1 |England|Championship            |
|E3 |England|League 2                |
|D1 |Germany|Bundesliga 1            |
|D2 |Germany|Bundesliga 2            |
|I2 |Italy  |Serie B                 |
|I1 |Italy  |Serie A                 |
|SP2|Spain  |La Liga Segunda Division|
|SP1|Spain  |La Liga Primera Division|
+---+-------+------------------------+

+------------------------+------------+------------+---------------------+------------------+
|League                  |Number Goals|Number Teams|first(Country, false)|Goals/Games       |
+------------------------+------------+------------+---------------------+------------------+
|Division 2              |1341        |38          |France               |0.9537695590327169|
|Primera Division        |746         

In [27]:
## Check the distinct Teams in those leagues (481) The new criteria has given us a wider search
Minor_Leagues.where(col("League").isin(Leagues_li)).select(col("HomeTeam")).distinct().head(30)
Minor_Leagues.where(col("League").isin(Leagues_li)).select(col("HomeTeam")).distinct().count()

481

In [28]:
#Make sure that Eredivisie is only in Netherlands

footballDF.where(col("League").isin("Eredivisie")).select(col("Country")).distinct().show()


+-----------+
|    Country|
+-----------+
|Netherlands|
+-----------+



In [29]:
# We make a list of Teams to analyze and we take out the Championship in England as it has teams of Premier League
Teams_Minor_Leagues_List = Minor_Leagues.where((col("League").isin(Leagues_li))&(col("League")!="Championship")).select(col("HomeTeam")).distinct().rdd.map(lambda x: x.HomeTeam).collect()

In [30]:
Teams_Minor_Leagues_List

['Antwerp',
 'Twente',
 'FC Voluntari',
 'Wisla',
 'Astra',
 'Avai',
 'Brage',
 'Harelbeke',
 'Sagan Tosu',
 'UC Dublin',
 'Cambuur',
 'Ceahlaul',
 'Kristiansund',
 'Nimes',
 'Helsingor',
 'Fortaleza',
 'AIK',
 'St Truiden',
 'Graafschap',
 'Samsunspor',
 'Roosendaal',
 'Santos',
 'AEK',
 'NAC Breda',
 'Toronto FC',
 'Nordsjaelland',
 'Mersin Idman Yurdu',
 'Athlone',
 'Goteborg',
 'KPV Kokkola',
 'TPS',
 'Levadeiakos',
 'Thrasyvoulos',
 'Atletico GO',
 'Mjondalen',
 'Atvidabergs',
 'Valenciennes',
 'Stromsgodset',
 'Besiktas',
 'Apollon',
 'FC Rapid Bucuresti',
 'Ilves',
 'Chicago Fire',
 'Kalamaria',
 'Mioveni',
 'Concordia',
 'Viitorul Constanta',
 'Nice',
 'Sepsi Sf. Gheorghe',
 'Atlanta United',
 'Arles',
 'Derry City',
 'Lommel',
 'Montpellier',
 'Minnesota United',
 'Yozgatspor',
 'Korona Kielce',
 'Xanthi',
 'Dijon',
 'Inter Turku',
 'Veracruz',
 'Wexford',
 'Beveren',
 'Iraklis',
 'Grenoble',
 'Goztep',
 'Mjallby',
 'Los Angeles Galaxy',
 'G-Osaka',
 'Vasco',
 'Antalyaspor',
 

In [31]:
## Checking if for those leagues the teams do not have Nulls in the Ocassions
Minor_Leagues.where(col("HomeTeam").isin(Teams_Minor_Leagues_List)).select("HomeTeam").where(col("HST").isNotNull()).distinct().show()

+------------+
|    HomeTeam|
+------------+
|     Antwerp|
|      Twente|
|       Nimes|
|  St Truiden|
|  Graafschap|
|         AEK|
|   NAC Breda|
| Levadeiakos|
|Valenciennes|
|    Besiktas|
|     Apollon|
|        Nice|
|       Arles|
| Montpellier|
|      Xanthi|
|       Dijon|
|    Grenoble|
|      Goztep|
| Antalyaspor|
|    Boulogne|
+------------+
only showing top 20 rows



In [32]:
# The first table shows how effective teams are regarding the goals vs ocassions at home.
# The second shows the same information but for teams playing away.
# So we need to look into the first column for the first table and second column for the second table

# Trabzonspor, Zwolle, Eupen, Utretcht and Waregem would be Teams to consider to look for precise strikers
# Waasland-Beveren, Heerenveen, Antalyasport and Aris would be interesting teams to check for strikers
# who have good numbers in away matches (dealing with pressure)


# Comment: We use coalesce to set to -1 whenever goals are not available in the dataset
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, sum, coalesce,avg

footballDF_2019.where(col("HomeTeam").isin(Teams_Minor_Leagues_List)).withColumn("EffectivityHome",coalesce(col("FTHG")/col("HST"),lit(-1.0))).withColumn("EffectivityAway",coalesce(col("FTAG")/col("AST"),lit(-1.0))).groupby("HomeTeam").agg((avg("EffectivityHome").alias("AVG_Eff_Home")),(avg("EffectivityAway").alias("AVG_Eff_Away"))).orderBy(col("AVG_Eff_Home").desc()).show()

footballDF_2019.where(col("AwayTeam").isin(Teams_Minor_Leagues_List)).withColumn("EffectivityHome",coalesce(col("FTHG")/col("HST"),lit(-1.0))).withColumn("EffectivityAway",coalesce(col("FTAG")/col("AST"),lit(-1.0))).groupby("HomeTeam").agg((avg("EffectivityHome").alias("AVG_Eff_Home")),(avg("EffectivityAway").alias("AVG_Eff_Away"))).orderBy(col("AVG_Eff_Away").desc()).show()




+----------------+-------------------+-------------------+
|        HomeTeam|       AVG_Eff_Home|       AVG_Eff_Away|
+----------------+-------------------+-------------------+
|     Trabzonspor|0.47775623069740714|0.34098972922502335|
|          Zwolle|0.47182539682539676| 0.3974206349206349|
|           Eupen|0.47023809523809523|0.31071428571428567|
|         Utrecht|0.45515873015873015|0.31646825396825395|
|         Waregem|  0.449808128379557| 0.2605442176870748|
|Sparta Rotterdam|0.44273504273504266| 0.3194139194139194|
|        Paris SG| 0.4339208410636982|0.16156462585034012|
|Asteras Tripolis|0.42009803921568634| 0.3326797385620915|
|      Buyuksehyr| 0.4179738562091504|0.20119047619047617|
|        Mouscron|0.41777777777777775|0.25904761904761897|
|          Twente| 0.4175824175824176|0.26520979020979024|
|            Gent|0.41713083213083213|0.15619047619047619|
|          Amiens| 0.4095238095238095| 0.3186507936507936|
|             AEK|0.40734126984126984|0.2018518518518518

In [33]:
footballDF_2019.where(col("HomeTeam").isin(Teams_Minor_Leagues_List)).withColumn("TacticalFoulsHome",coalesce((col("HY")+col("HR"))/col("HF"),lit(-1.0))).withColumn("TacticalFoulsAway",coalesce((col("AY")+col("AR"))/col("AF"),lit(-1.0))).groupby("HomeTeam").agg((avg("TacticalFoulsHome").alias("AVG_TacFoul_Home")),(avg("TacticalFoulsAway").alias("AVG_TacFoul_Away"))).orderBy(col("AVG_TacFoul_Home").desc()).show()

footballDF_2019.where(col("AwayTeam").isin(Teams_Minor_Leagues_List)).withColumn("TacticalFoulsHome",coalesce((col("HY")+col("HR"))/col("HF"),lit(-1.0))).withColumn("TacticalFoulsAway",coalesce((col("AY")+col("AR"))/col("AF"),lit(-1.0))).groupby("HomeTeam").agg((avg("TacticalFoulsHome").alias("AVG_TacFoul_Home")),(avg("TacticalFoulsAway").alias("AVG_TacFoul_Away"))).orderBy(col("AVG_TacFoul_Away").desc()).show()

# We need to look into the first column for the first table and second column for the second table.


## If we look into the tables, there are some potential Teams to go delve into their players, as they
# may have a good striker with a strong tactical Midfield commiting Faults but avoiding many cards.
# Those teams could be Besiktas, Valenciennes or Goztep, appearing in top10 for both home and away games.
# Besides, Zwolle is a versatile team, figuring out in the previous tables and the following ones
# in Home and Away Games. Potential players could be investigated by the club for next Seasson.



# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HFKC = Home Team Free Kicks Conceded
# AFKC = Away Team Free Kicks Conceded
# HO = Home Team Offsides
# AO = Away Team Offsides
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards
# HR = Home Team Red Cards
# AR = Away Team Red Cards

+----------------+-------------------+-------------------+
|        HomeTeam|   AVG_TacFoul_Home|   AVG_TacFoul_Away|
+----------------+-------------------+-------------------+
|        Besiktas| 0.2508150045688745|0.22728830387726767|
|    Valenciennes| 0.2441667051435959|0.18215960626674907|
|          Xanthi|0.24058732568600985|0.21720744533244532|
|  Genclerbirligi| 0.2365719237964731|0.17225223834220374|
|          Goztep|0.23241369913196538|0.25975842688645456|
|     For Sittard|0.21903972096279786|0.19879891350479584|
|       Konyaspor| 0.2178643252172664| 0.1571479060711441|
|       Panionios|0.21654314170898248| 0.2002367941893997|
|       Gaziantep|0.21478464927290275| 0.3071399274959647|
|Asteras Tripolis|0.21477703919149604|0.20941870357732678|
|     Kayserispor| 0.2142417195504065|0.23450659298410162|
|        Waalwijk|0.21339927503895112| 0.1517130305591844|
|          Larisa|0.21020684226839756|0.18949312496243048|
|      St Truiden|0.20955134047239313| 0.135362554112554

In [34]:
# We can locate in which leagues and Divisions this team has played
footballDF.where(col("HomeTeam")=="Goztep").select("League","Div").distinct().show()

+-------------+---+
|       League|Div|
+-------------+---+
|Futbol Ligi 1| T1|
+-------------+---+



In [35]:
# We can locate in which leagues and Divisions this team has played

footballDF.where(col("HomeTeam")=="Besiktas").select("League","Div").distinct().show()

# Since there are some leagues where they played, let's see this year to get an idea of the market value
# of their players

footballDF.where((col("HomeTeam")=="Besiktas")&(col("Season")=="2019/2020")).select("League","Div").distinct().show()

# The team is in 3rd Division, and may have kept some players from the previous season, and their price
# may be not too high.

+-------------+---+
|       League|Div|
+-------------+---+
|Futbol Ligi 1| T1|
+-------------+---+

+-------------+---+
|       League|Div|
+-------------+---+
|Futbol Ligi 1| T1|
+-------------+---+

